In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import scipy

# empty data frames
asset_prices_df = pd.DataFrame()
asset_pred_1_df = pd.DataFrame()
asset_pred_2_df = pd.DataFrame()
asset_pred_3_df = pd.DataFrame()

asset_prices_list = []
asset_pred_1_list = []
asset_pred_2_list = []
asset_pred_3_list = []

fin_weights_list = [np.repeat(1 / 9, 9)]

#shares outstanding
#just change path
shares = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/case3data/Shares Outstanding.csv")
shares_arr = shares.iloc[[0],[1,2,3,4,5,6,7,8,9]].to_numpy()

#tau - tuning constant
tau = 0.05

#risk aversion parameter 2.15 to 2.65, just to simplify
rap = 2.4

#P - picking matrix
P = np.array(
  [
      [1, 0, 0, 0, 0, 0, 0, 0, 0],
      [0, 1, 0, 0, 0, 0, 0, 0, 0],
      [0, 0, 1, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 1, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 1, 0, 0, 0, 0],
      [0, 0, 0, 0, 0, 1, 0, 0, 0],
      [0, 0, 0, 0, 0, 0, 1, 0, 0],
      [0, 0, 0, 0, 0, 0, 0, 1, 0],
      [0, 0, 0, 0, 0, 0, 0, 0, 1],
  ]
)

#previous sharpe
p_s = 0

#to think, update each time considering previous weight values?

def allocate_portfolio(asset_prices, asset_price_predictions_1, 
                       asset_price_predictions_2,
                       asset_price_predictions_3):
  
    #update dataframes
    asset_prices_list.append(asset_prices)
    asset_prices_df = pd.DataFrame(asset_prices_list)
    
    asset_pred_1_list.append(asset_price_predictions_1)
    asset_pred_1_df = pd.DataFrame(asset_pred_1_list)

    asset_pred_2_list.append(asset_price_predictions_2)
    asset_pred_2_df = pd.DataFrame(asset_pred_2_list)

    asset_pred_3_list.append(asset_price_predictions_3)
    asset_pred_3_df = pd.DataFrame(asset_pred_3_list)


    #check that there are at least 2 days worth of data
    if (len(asset_prices_list) < 3):
      n_assets = len(asset_prices)
      weights = np.repeat(1 / n_assets, n_assets)
      return weights
    
    #create return series - output DataFrame
    #to consider for improvement: using log instead of raw return
    ret_real = convert_returns(asset_prices_df)
    ret_df1 = convert_returns(asset_pred_1_df)
    ret_df2 = convert_returns(asset_pred_2_df)
    ret_df3 = convert_returns(asset_pred_3_df)

    #var-covar matrices - output NumPy array
    covar_real = get_varcovar(ret_real)
    covar_1 = get_varcovar(ret_df1)
    covar_2 = get_varcovar(ret_df2)
    covar_3 = get_varcovar(ret_df3)

    #final day idx
    idx = len(asset_prices_list) - 1

    #final day's price
    fin_price_real = one_day_price(asset_prices_df, idx)
    fin_price_1 = one_day_price(asset_pred_1_df, idx)
    fin_price_2 = one_day_price(asset_pred_2_df, idx)
    fin_price_3 = one_day_price(asset_pred_3_df, idx)

    #implied returns for curr day
    imp_real = implied_market_ret(fin_price_real, covar_real, rap, shares_arr)
    imp_1 = implied_market_ret(fin_price_1, covar_1, rap, shares_arr)
    imp_2 = implied_market_ret(fin_price_2, covar_2, rap, shares_arr)
    imp_3 = implied_market_ret(fin_price_3, covar_3, rap, shares_arr)

    #confidence matrix
    conf_real = conf_mat(tau, P, covar_real)
    conf_1 = conf_mat(tau, P, covar_1)
    conf_2 = conf_mat(tau, P, covar_2)
    conf_3 = conf_mat(tau, P, covar_3)

    #view for curr day
    view_1 = one_day_view(ret_df1, idx - 1)
    view_2 = one_day_view(ret_df2, idx - 1)
    view_3 = one_day_view(ret_df3, idx - 1)

    #latest real price
    rp = one_day_price(asset_prices_df, idx)

    #expected returns
    er1 = exp_ret(tau, covar_1, P, conf_1, shares_arr, view_1, rp)
    er2 = exp_ret(tau, covar_2, P, conf_2, shares_arr, view_2, rp)
    er3 = exp_ret(tau, covar_3, P, conf_3, shares_arr, view_3, rp)

    #individual weights
    fw1 = fin_weights(tau, covar_1, er1)
    fw2 = fin_weights(tau, covar_2, er2)
    fw3 = fin_weights(tau, covar_3, er3)

    fw1 = normalize(fw1)
    fw2 = normalize(fw2)
    fw3 = normalize(fw3)

    #average weights
    fin_res = (fw1 + fw2 + fw3)
    fin_res = np.transpose(fin_res)[0]
    #print(fin_res)
    #fin_res = normalize(fin_res)
    if len(fin_weights_list) > 9:
      fin_weights_list.clear()
      fin_weights_list.append(np.repeat(1 / 9, 9))

    start = fin_weights_list[0]
    for f in range(1, len(fin_weights_list)):
      start = start + fin_weights_list[f]

    last_ret = start * one_day_price(ret_real, idx - 1)
    curr_ret = fin_res * one_day_price(ret_real, idx - 1)
    sharpe1 = np.mean(last_ret) / np.std(last_ret)
    sharpe2 = np.mean(curr_ret) / np.std(curr_ret)

    if sharpe2 > sharpe1:
      fin_weights_list.append(fin_res)
      # start = fin_weights_list[0]
      # for f in range(1, len(fin_weights_list)):
      #   start = start + fin_weights_list[f]
    else:
      fin_res = start

    #check sharpe ratio
    curr_ret = fin_res * one_day_price(ret_real, idx - 1)
    fin_sharpe = np.mean(curr_ret) / np.std(curr_ret)

    print(fin_sharpe)
    
    return fin_res


In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/case3data/Predicted Testing Data Analyst 1.csv")
df2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/case3data/Predicted Testing Data Analyst 2.csv")
df3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/case3data/Predicted Testing Data Analyst 3.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/case3data/Acutal Testing Data.csv")

df1_norm = np.delete(df1.to_numpy(), 0, 1) 
df2_norm = np.delete(df2.to_numpy(), 0, 1) 
df3_norm = np.delete(df3.to_numpy(), 0, 1) 
test_norm = np.delete(test_data.to_numpy(), 0, 1) 

avg_sharp = 0

for i in range(df1_norm.shape[0]):
  #for i in range(50):
  inp1 = df1_norm[i].tolist()
  inp2 = df2_norm[i].tolist()
  inp3 = df3_norm[i].tolist()
  test_inp = test_norm[i].tolist()
  x = allocate_portfolio(test_inp, inp1, inp2, inp3)
  avg_sharp += x

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


0.5369851518323919
-0.5284781566085339
-0.08442421495083995
0.20319440751829512
0.5806971812664322
0.3386144221989143
-0.09087853848113699
0.07201981384070096
0.316668937611905
0.156289398748508
0.7026519758320197
-1.140996655167615
2.485165550853381
-0.8581516054285846
1.5468068275503377
0.14201214235011003
-0.8765265039478499
-0.582744473110602
-0.3516620547604786
-0.4669745515296412
0.37789883059436835
0.2364078103895237
0.1228144534982846
0.45663791323536945
0.8810374040457521
0.0828239253955128
0.3595536537814943
0.5726488733907741
0.3029977145393857
0.13833174792245498
-0.1223935632833895
1.0084085988803242
0.07935437937354596
0.23913698264935257
0.056174665012374815
-0.15447540403845939
0.5470802179859621
-0.09228458631843016
-0.22676831378907622
0.39806373140045054
0.8304530757913792
-0.348898490039457
0.22129804460916702
-0.062884539890587
0.807515023155873
0.4575422842979041
0.6658185600663871
0.6046008832404236
0.2189223128028073
-0.2990418141625864
0.4610698342945803
0.3583

KeyboardInterrupt: ignored

In [ ]:
def convert_returns(data):
  return_series = []
  for i in range(1, data.shape[0]):
    row_i = data.iloc[[i],[0,1,2,3,4,5,6,7,8]].to_numpy()
    row_j = data.iloc[[i-1],[0,1,2,3,4,5,6,7,8]].to_numpy()
    #return_series.append(row_i[0] / row_j[0] - 1)
    return_series.append(np.log(row_i[0] / row_j[0]))
  return pd.DataFrame(return_series)

In [ ]:
#var covar matrix (return numpy 2d array 9x9)
def get_varcovar(data):  
  #find average of returns
  avg_arr = []
  for i in range(9):
    avg_arr.append(data[i].mean())
  avg_arr = np.array(avg_arr)

  #excess returns
  excess = []
  for row in data.to_numpy():
    excess.append(row - avg_arr)
  excess = np.array(excess)

  #final step
  t_excess = np.transpose(excess)
  div = data.shape[0] - 1
  var_covar = np.matmul(t_excess, excess)
  var_covar = var_covar / div
  return var_covar

In [ ]:
#calculate market implied returns
def implied_market_ret(day_data, var_covar, delta, shares_arr):

  #calculate market-cap weight ?? for each day

  #for day one (index: 1)
  #one_day = df1.iloc[[1],[1,2,3,4,5,6,7,8,9]].to_numpy()
  mcap_w = day_data * shares_arr
  mcap_w = mcap_w / np.sum(mcap_w)
  mcap_w = np.transpose(mcap_w)

  # final step
  implied_ret = np.matmul((delta * var_covar), mcap_w) 
  return implied_ret

In [ ]:
def one_day_price(data, day):
  #day - index start with 1
  return data.iloc[[day],[0,1,2,3,4,5,6,7,8]].to_numpy()

In [ ]:
def one_day_view(data, day):
  #data should be return series
  #day - index start with 0
  return data.to_numpy()[day].reshape(-1, 1)

In [ ]:
def conf_mat(const, pick, covar):
  #confidence matrix
  #tau * (picking matrix * covariance matrix * picking transposed)
  return const * np.matmul(np.matmul(pick, covar), np.transpose(pick))

In [ ]:
def exp_ret(const, covar, pick, conf, shares_arr, view, real_price):
  #one day view (latest day?)
  #real_price (corresponding day)
  part1 = const * covar
  try:
    part1 = np.linalg.inv(part1)
  except:
    part1 = np.linalg.lstsq(part1, pick, rcond=None)
    part1 = part1[0]

  try:
    inv_conf = np.linalg.inv(conf)
  except:
    inv_conf = np.linalg.lstsq(conf, pick, rcond=None)[0]

  part1 += np.matmul(np.matmul(np.transpose(pick), inv_conf), pick) 

  try:
    part1 = np.linalg.inv(part1)
  except:
    part1 = np.linalg.lstsq(part1, pick, rcond=None)
    part1 = part1[0]

  part2 = const * covar

  try:
    part2 = np.linalg.inv(part2)
  except:
    part2 = np.linalg.lstsq(part2, pick, rcond=None)
    part2 = part2[0]

  imp = implied_market_ret(real_price, covar, const, shares_arr)
  part2 = np.matmul(part2, imp)
  part2 += np.matmul(np.matmul(np.transpose(pick), inv_conf), view)
  er = np.matmul(part1, part2)
  return er

In [ ]:
def fin_weights(const, covar, er):
  #er - expected return
  w1 = const * covar

  try:
    w1 = np.linalg.inv(w1)
  except:
    w1 = np.linalg.lstsq(w1, P, rcond=None)[0]

  w1 = np.matmul(w1, er)
  w1 *= 2
  return w1

In [ ]:
def normalize(arr):
  l1 = np.linalg.norm(arr)
  arr = arr / l1
  return arr